In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#loading data
df_female = pd.read_csv('test_data2/df_female.csv')
df_age = pd.read_csv('test_data2/df_age.csv')
df_SFI = pd.read_csv('test_data2/df_SFI.csv')
df_uti = pd.read_csv('test_data2/df_uti.csv')
df_admissions = pd.read_csv('test_data2/df_admissions.csv')
df_FIM_total = pd.read_csv('test_data2/df_FIM_total.csv')
df_acute_days = pd.read_csv('test_data2/df_acute_days.csv')
df_brain_injury = pd.read_csv('test_data2/df_brain_injury_mod.csv')

#Removing extra column
df_female = df_female.loc[:, ~df_female.columns.str.contains('^Unnamed')]
df_age = df_age.loc[:, ~df_age.columns.str.contains('^Unnamed')]
df_SFI = df_SFI.loc[:, ~df_SFI.columns.str.contains('^Unnamed')]
df_uti = df_uti.loc[:, ~df_uti.columns.str.contains('^Unnamed')]
df_admissions = df_admissions.loc[:, ~df_admissions.columns.str.contains('^Unnamed')]
df_FIM_total = df_FIM_total.loc[:, ~df_FIM_total.columns.str.contains('^Unnamed')]
df_acute_days = df_acute_days.loc[:, ~df_acute_days.columns.str.contains('^Unnamed')]
df_brain_injury = df_brain_injury.loc[:, ~df_brain_injury.columns.str.contains('^Unnamed')]


In [ ]:
#Creating uti df with negative cases as well
merged = pd.merge(df_age, df_uti, on=['ID'], how='left')

merged = merged[['ID','value']]

#turning UVI-neg cases from nan to 0
where_are_NaNs = np.isnan(merged)
merged[where_are_NaNs] = 0

#checking
print(merged)

#Changing from floats to int
merged['value'] = merged['value'].astype('int')

merged = pd.merge(merged, df_uti, on=['ID', 'value'], how='left')

print(merged)

In [ ]:
#Important to call predictors static, dynamic or text, otherwise they will be skipped because the function will not know how to handle them!
#Make a disclaimer message if there is such a df in the predictor dict
predictor_dict = {'df_SFI_text':df_SFI, 'df_age_static':df_age, 'df_female_static':df_female}

In [ ]:
from preprocessing import data_split

predictor_dict_train, predictor_dict_test, y_train_df, y_test_df, df_admissions_train, df_admissions_test = data_split(df_outcome = merged, predictor_dict = predictor_dict, df_admissions = df_admissions, test_size = 0.3)

In [ ]:
from tfidf import tf_idf

df_tfidfvect_train, df_tfidfvect_test = tf_idf(predictor_dict_train["df_SFI_text"], predictor_dict_test["df_SFI_text"])

# #Bør tal-tokens slettes?

In [ ]:
##Might be a stupid question, but since I use a pretrained embeddings model, I guess I should treat the test and train data in the same way?

#from sentence_embeddings import sentence_embeddings

#df_sentence_train = sentence_embeddings(df_free_text = predictor_dict_train["df_SFI"], transformer_model = 'encoder-large-v1')

#df_sentence_test = sentence_embeddings(df_free_text = predictor_dict_test["df_SFI"], transformer_model = 'encoder-large-v1')

In [ ]:
from preparing_specs import preparing_specs



X_final_train_tfidf, y_final_train_tfidf =  preparing_specs(embeddings = df_tfidfvect_train,
                                                predictor_dict = predictor_dict_train,
                                                prediction_times = df_admissions_train, 
                                                outcome_df = y_train_df, 
                                                embedding_type ="tfidf",
                                                lookbehind = 4, 
                                                lookahead = 3)


# X_final_train_transf, y_final_train_transf=  preparing_specs(embeddings = df_sentence_train,
#                                                 predictor_dict = predictor_dict_train,
#                                                 prediction_times = df_admissions_train, 
#                                                 outcome_df = y_train_df, 
#                                                 embedding_type ="transformer",
#                                                 lookbehind = 4, 
#                                                 lookahead = 3)
